# HEI Integration statistics 

In [ ]:
# Packages
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np
import seaborn as sns
import json


In [ ]:
# domain regex for extraction
domain_regex = r'(?<=@)[^@\s]+' #r'(?<=@)[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'

In [ ]:
# Commercial pool
commercial_pool = ['aol', 'gmail.com', 'googlemail.com', 'gmx', 'web.de', 'icloud', 't-online', 'hotmail', 'yahoo', 'outlook', 'posteo', 'freenet', 'me.com', 'mailbox.org', 'live', 'arcor', 'mail.de', 'online.de', 'bluewin.ch', 'proton', 'qq.com', 'ymail', '163.com', 'magenta', 'netcologne', 'duck', 'pm.me', 'tutanota', 'mail.ru', 'mac.com', 'unitybox', 'telekom', 'GMAIL.COM', 'onlinehome', 'chello', 'rocketmail', 'msn', 'GMAIL.com', 'jioso.com', 'mozmail.com']
commercial_regex = '|'.join(commercial_pool)

In [ ]:
# possible edu domain regex
edu_regex = '|'.join(['fh', 'hs', 'uni', 'tu-', 'th-' 'stud', 'student', 'students', 'hochschul', 'uni', 'dir.de', 'uka', 'eah-', 'dhbw', 'tib', 'ils', 'usb', 'tu-', 'htw', 'ku.de', 'ph', 'upb', 'lmu', 'h-ka', 'hwg', 'h-brs', 'rub', '.edu', 'hft', 'mni', 'kit', 'bth', 'medical', 'example', 'campus'])


### Import the ground truth as well as the maste_list from the previous round
* The approach is based on the files "all_email_provider_domains.txt", "world_universities_and_domains.json" and "german-heis.csv" on the Github public repos
* https://github.com/jacksonbarreto/Higher-Education-Institutions-in-Germany-Dataset/blob/main/data%20source/german-heis.csv
* https://github.com/Hipo/university-domains-list?utm_source=chatgpt.com
* https://gist.github.com/humphreybc/d17e9215530684d6817ebe197c94a76b or https://gist.github.com/ammarshah/f5c2624d767f91a7cbdc4e54db8dd0bf?permalink_comment_id=4449999

In [ ]:
# load files

# commercial providers
with open('C:/Users/ywa/Documents/Data/UserReport/all_email_provider_domains.txt', 'r') as f:
    commercial_providers = set(f.read().splitlines())

# world university domains
try:
    with open('C:/Users/ywa/Documents/Data/UserReport/world_universities_and_domains.json', 'r', encoding='utf-8') as f:
        world_universities = json.load(f)
except UnicodeDecodeError:
    with open('C:/Users/ywa/Documents/Data/UserReport/world_universities_and_domains.json', 'r', encoding='ISO-8859-1') as f:
        world_universities = json.load(f)

uni_domains = pd.DataFrame(world_universities)
uni_domains['web_pages'] = [x[0] for x in  uni_domains['web_pages']]

uni_domains['clean_domain'] = uni_domains['web_pages'].str.replace("http://", "").str.replace("https://", "").str.replace('/', '').str.replace('www.', '')

uni_domains_wo_de = uni_domains[uni_domains['country'] != 'Germany']

#uni_domains_de = uni_domains[uni_domains['country'] == 'Germany']

# German-heis
german_heis = pd.read_excel('C:/Users/ywa/Documents/Data/UserReport/german-heis.xlsx', sheet_name = 'Final_sheet')
german_heis['clean_domain'] = german_heis['Url'].str.replace('www.', '')
#german_heis['country'] = 'Germany'

hei_domains = pd.concat([german_heis[['Name', 'clean_domain', 'country']].rename(columns = {'Name' : 'name'}), uni_domains[['name', 'clean_domain', 'country']]]).drop_duplicates(subset = ['clean_domain'], keep = 'last').reset_index(drop = True)


In [ ]:
# load master list
master_path = 'C:/Users/ywa/Documents/Data/UserReport/MasterList_Update.csv'
master_list = pd.read_csv(master_path, sep = ',', usecols = ['domain', 'country', 'provider_type', 'provider'], encoding = 'utf8')

In [ ]:
# clean the master list
master_list.head()
master_list['domain'] = master_list['domain'].str.strip().str.replace(r'\s+', '', regex=True).str.normalize('NFKC')
master_list['country'] = master_list['country'].str.strip()
master_list['provider'] = master_list['provider'].str.strip()


In [ ]:
master_list_dup = master_list[master_list['domain'].duplicated()]
master_list_dup

In [ ]:
# Sanity check master list
master_list[master_list.duplicated(subset = ['domain', 'provider'])]

print(len(master_list))
master_list = master_list.drop_duplicates(subset = ['domain', 'provider'])

print(len(master_list))

master_list.head()

### Drupal Data Analysis

In [ ]:
# load data
drupal_path = 'C:/Users/ywa/Documents/Data/UserReport/Jan_2026_Moodle/drupal_Jan_email.xlsx'
drupal = pd.read_excel(drupal_path, sheet_name = 'emails')

In [ ]:
# extract emails
email_drupal = drupal['E-Mail-Adresse']
print(len(email_drupal))

email_drupal.head()


In [ ]:
# email sanity check
problematic_emails_d = email_drupal[
    ~email_drupal.apply(lambda x: isinstance(x, str)) |
    ~email_drupal.astype(str).str.contains('@', na=False)
]
print(problematic_emails_d)
print(len(problematic_emails_d))

In [ ]:
# drop problematic entries

# if missing value
email_drupal = email_drupal.dropna()

print(len(email_drupal))

In [ ]:
# domain extraction, output is a df with 'domain' and 'count' columns
drupal_regex = email_drupal.astype(str).str.findall(domain_regex)
drupal_domains = [item[0] if item else None for item in drupal_regex]
drupal_domains = pd.Series(drupal_domains).str.lower()
drupal_domains = pd.Series(drupal_domains).str.strip().str.replace(r'\s+', '', regex=True).str.normalize('NFKC')
drupal_domains = pd.DataFrame(drupal_domains.value_counts())
drupal_domains = drupal_domains.reset_index()
drupal_domains.columns = ['domain', 'count']
drupal_domains

In [ ]:
# quick sanity check of the sum
print(drupal_domains['count'].sum())
print(len(email_drupal))


In [ ]:
# merge master list ', 
drupal_merge = drupal_domains.merge(master_list, on = 'domain', how = 'left', validate = 'one_to_one')
print(drupal_merge.duplicated(subset = ['domain', 'provider']).sum())

drupal_matched_count = drupal_merge['provider_type'].notna().sum()

print(f'the first round matched record is: {drupal_matched_count}, the total record of the new list is {len(drupal_domains)}, the total record of the master list is {len(master_list)}')

# sanity check
print(drupal_merge['count'].sum())
print(drupal_merge.shape)

# check on the merged value
drupal_merge.head()

In [ ]:
# standardise DHBW entries
dhbw_regex = 'dhbw'
dhbw_mask = drupal_merge['domain'].str.contains(dhbw_regex)
drupal_merge.loc[dhbw_mask, ['country', 'provider_type', 'provider']] = ['Germany', 2.0, 'DHBW']

# DHBW sanity check
drupal_merge.loc[dhbw_mask]

# change record of f269323f-fa99-4102-81fa-2e6ee79d13e8.ilias and example.com
#unique_domain_copy.loc[unique_domain_copy['domain'] == 'f269323f-fa99-4102-81fa-2e6ee79d13e8.ilias', ['provider']] = 'Landesärztekammer BaWü'
#unique_domain_copy.loc[unique_domain_copy['domain'] == 'example.com', ['provider']] = 'FernUniversität in Hagen'


In [ ]:
# if there're not matched domains
drupal_no_match = drupal_merge[drupal_merge['provider_type'].isna()].copy()
drupal_no_match


In [ ]:
# loop over on the ground truth

# Commercial providers
drupal_no_match['provider_type'] = drupal_no_match['domain'].apply(lambda x : 0 if x in commercial_providers else None)
drupal_no_match['provider'] = drupal_no_match['provider_type'].apply(lambda x : 0 if x == 0 else None)
drupal_mask_commercial = drupal_no_match['domain'].str.contains(commercial_regex)
drupal_no_match.loc[drupal_mask_commercial & drupal_no_match['provider_type'].isna(), ['provider_type', 'provider']] = [0, 0]

# HEI providers
for _, row in hei_domains.sort_values(by = 'clean_domain', key = lambda x : x.str.len(), ascending = False).iterrows():
    new_dmask_d = drupal_no_match['domain'].str.endswith(row['clean_domain'], na = False)
    drupal_no_match.loc[new_dmask_d & drupal_no_match['provider_type'].isna(), ['provider_type', 'provider', 'country']] = [2, row['name'], row['country']]
    
drupal_no_match['provider_type'].fillna(1, inplace = True)
drupal_no_match['provider'].fillna(0, inplace = True)


In [ ]:
# Sanity check-point --> the not-filled data should be filled 
print(drupal_no_match['provider_type'].isna().sum())


In [ ]:
# repair some mis-matches <post hoc>
# Frankfurt University of Applied Sciences
fra_uas_regex = 'fra-uas.de'
fra_uas_mask = drupal_merge['domain'].str.contains(fra_uas_regex)
drupal_merge.loc[fra_uas_mask, ['country', 'provider_type', 'provider']] = ['Germany', 2.0, 'Frankfurt University of Applied Sciences']


# the missing ones
drupal_merge.loc[drupal_merge['domain'] == 'stud.hn.de', ['country', 'provider_type', 'provider']] = ['Germany', 2.0, 'Hochschule Niederrhein']
drupal_merge.loc[drupal_merge['domain'] == 'stud.fra-uas.de', ['country', 'provider_type', 'provider']] = ['Germany', 2.0, 'Frankfurt University of Applied Sciences']
drupal_merge.loc[drupal_merge['domain'] == 'stud.ph-gmuend.de', ['country', 'provider_type', 'provider']] = ['Germany', 2.0, 'PH Schwäbisch Gmünd']
drupal_merge.loc[drupal_merge['domain'] == 'stud.hs-furtwangen.de', ['country', 'provider_type', 'provider']] = ['Germany', 2.0, 'Hochschule Furtwangen']
drupal_merge.loc[drupal_merge['domain'] == 'tha.de', ['country', 'provider_type', 'provider']] = ['Germany', 2.0, 'Hochschule Augsburg']
drupal_merge.loc[drupal_merge['domain'] == 'ukbonn.de', ['country', 'provider_type', 'provider']] = ['Germany', 2.0, 'Universitätsklinikum Bonn']
drupal_merge.loc[drupal_merge['domain'] == 'iu.org', ['country', 'provider_type', 'provider']] = ['Germany', 2.0, 'International University']
drupal_merge.loc[drupal_merge['domain'] == 'stud.fra-uas.de', ['country', 'provider_type', 'provider']] = ['Germany', 2.0, 'Frankfurt University of Applied Sciences']
drupal_merge.loc[drupal_merge['domain'] == 'dacs-bund.de', ['country', 'provider_type', 'provider']] = ['Germany', 2.0, 'Hochschule des Bundes für öffentliche Verwaltung']
drupal_merge.loc[drupal_merge['domain'] == 'khm.de', ['country', 'provider_type', 'provider']] = ['Germany', 2.0, 'Kunsthochschule für Medien Köln']
drupal_merge.loc[drupal_merge['domain'] == 'itu.edu.tr', ['country', 'provider_type', 'provider']] = ['Turkey', 2.0, 'Instanbul Technical University']
drupal_merge.loc[drupal_merge['domain'] == 'aau.at' , ['country', 'provider_type', 'provider']] = ['Austria', 2.0, 'Universität Klagenfurt']
drupal_merge.loc[drupal_merge['domain'] == 'hcw.ac.at' , ['country', 'provider_type', 'provider']] = ['Austria', 2.0, 'FH Campus Wien']

In [ ]:
# sanity check
drupal_merge[drupal_merge['domain'] == 'khm.de']

In [ ]:
# remained possible edu domains
drupal_edu_candidate = drupal_no_match[(drupal_no_match['provider_type'] == 1) & (drupal_no_match['domain'].str.contains(edu_regex))]
print(drupal_edu_candidate.shape)
print(drupal_edu_candidate)

In [ ]:
# export potential candidates --> do not overwrite!!!
#drupal_edu_candidate.to_csv("C:/Users/ywa/Documents/Data/UserReport/test_potential_educational_domains_drupal.csv", index=False)
#print("Filtered domains saved for manual review.") 

In [ ]:
# make a copy
drupal_merge_copy = drupal_merge.copy()
print(drupal_merge_copy.shape)


In [ ]:
# Merge maual filtered record with the existing list
reviewed_edu_drupal = pd.read_csv("C:/Users/ywa/Documents/Data/UserReport/test_potential_educational_domains_drupal.csv")
# in case encoding='latin-1', sep = ';'
print(reviewed_edu_drupal.shape)

print(drupal_merge.shape)

# check dupliactes before merge
print(drupal_merge_copy[drupal_merge_copy.duplicated(subset = 'domain')])

# concatenate two dataframes
drupal_merge_copy = pd.concat([drupal_merge_copy, reviewed_edu_drupal], axis = 0)

print(drupal_merge_copy.shape)

print(drupal_merge_copy[drupal_merge_copy.duplicated(subset = ['domain', 'count'])])

print(drupal_merge_copy.shape)

drupal_merge_copy = drupal_merge_copy.drop_duplicates(subset = ['domain', 'count'], keep = 'last').reset_index(drop = True)

print(drupal_merge_copy.shape)

In [ ]:
# NaNs check
print(drupal_merge_copy.isna().sum())

print(drupal_merge_copy[drupal_merge_copy['provider_type'].isna()])

# Relpace NaNs
drupal_merge_copy['country'].fillna('unknown', inplace = True)
drupal_merge_copy['provider_type'].fillna(1.0, inplace = True)
drupal_merge_copy['provider'].fillna(0, inplace = True)

print(drupal_merge_copy.isna().sum())


In [ ]:
# sanity check! 
drupal_merge_copy['count'].sum()

In [ ]:
# Update the master list if there are domain updates
print(len(master_list))
print(len(drupal_merge_copy))

# update master list
master_update_d = pd.concat([master_list, drupal_merge_copy]).drop_duplicates(subset = ['domain'], keep = 'first').reset_index(drop = True)
print(len(master_update_d))

# for excel export
#master_update_d.to_excel("C:/Users/ywa/Documents/Data/UserReport/MasterList_Update.xlsx", index=False)

# for csv export --> used for next round
#master_update_d.to_csv("C:/Users/ywa/Documents/Data/UserReport/MasterList_Update.csv", index=False)


## Analysis!

In [ ]:
# count the number of unique domains
print(f'There are {len(drupal_merge_copy)} unique domains on Drupal')
drupal_merge_copy.head()

In [ ]:
# number of accounts based on the provider type
drupal_ptype_count = pd.DataFrame(drupal_merge_copy.groupby('provider_type')['count'].sum().sort_values(ascending = False))

print(drupal_ptype_count)

drupal_ptype_count.index = ['Commercial', 'HEIs', 'Others']
drupal_ptype_count['ratio'] = round((drupal_ptype_count['count'] / drupal_ptype_count['count'].sum()) * 100, 2)

drupal_ptype_count['count'].plot.bar(rot = 0)

print(drupal_ptype_count)


In [ ]:
# sanity check (total account)
drupal_ptype_count['count'].sum()

In [ ]:
# counting the number of domains in different categories
drupal_unique_domains = drupal_merge_copy['provider_type'].value_counts()
drupal_unique_domains = pd.DataFrame(drupal_unique_domains)

print(drupal_unique_domains)

drupal_unique_domains.index = ['Other', 'HEIs', 'Commercial']
drupal_unique_domains.columns = ['count']
drupal_unique_domains['ratio'] = round((drupal_unique_domains['count'] / drupal_unique_domains['count'].sum()) * 100, 2)
print(drupal_unique_domains)

drupal_unique_domains['count'].plot.bar(rot = 0)

In [ ]:
# sanity check (unique domain)
drupal_unique_domains['count'].sum()


In [ ]:
# HEI distribution in DACH region
drupal_de_heis = drupal_merge_copy[(drupal_merge_copy['country'] == 'Germany') & (drupal_merge_copy['provider_type'] == 2)]
drupal_at_heis = drupal_merge_copy[(drupal_merge_copy['country'] == 'Austria') & (drupal_merge_copy['provider_type'] == 2)]
drupal_ch_heis = drupal_merge_copy[(drupal_merge_copy['country'] == 'Switzerland') & (drupal_merge_copy['provider_type'] == 2)]
                                                                                      
print(f'There are {drupal_de_heis.shape[0]} unique domains from german HEIs')
print(f'There are {drupal_at_heis.shape[0]} unique domains from austrian HEIs')
print(f'There are {drupal_ch_heis.shape[0]} unique domains from swiss HEIs')

In [ ]:
# The number of accounts in HEIs
drupal_unique_heis = drupal_merge_copy[drupal_merge_copy['provider_type'] == 2.0]
drupal_unique_heis = pd.DataFrame(drupal_unique_heis.groupby('provider')['count'].sum().sort_values(ascending = False))

print(drupal_unique_heis['count'].sum())

drupal_unique_heis.head(50)


In [ ]:
# Select HEI with more than 100 accounts
drupal_integration_edu = drupal_unique_heis[drupal_unique_heis['count'] >= 100]
print(f'There are {len(drupal_integration_edu)} HEIs that have possibly integrated KIC content in the curriculum, which is {len(drupal_integration_edu) /len(drupal_unique_heis) * 100} percent of the total HEIs on KIC')


In [ ]:
# the number of accounts in the integrated HEI
drupal_integration_edu['count'].sum()

In [ ]:
# Geographic distribution of the HEI accounts
drupal_hei_country = drupal_merge_copy[(drupal_merge_copy['provider_type'] == 2.0)]
drupal_hei_ncountry = drupal_hei_country['country'].nunique()
print(f'The HEI users from {drupal_hei_ncountry} countries have registered on our platform.')

drupal_hei_countrycount = pd.DataFrame(drupal_hei_country.groupby('country')['count'].sum().sort_values(ascending = False))
print(drupal_hei_countrycount.head(10))


In [ ]:
# Geographic distribution of the German HEIS (federal states)

# match federal states by provider

####### with german_heis list#######
#de_unique_heis_drupal = drupal_de_heis.merge(german_heis[['Name', 'Region']], left_on = 'provider', right_on = 'Name',  how = 'left').rename(columns = {'Region' : 'state'})

#de_unique_heis_drupal = de_unique_heis_drupal.drop('Name', axis = 1)

#de_unique_heis_drupal['state'].isnull().sum()

###### with the provider-state combination from the first round ######
de_regions_drupal = pd.read_csv('C:/Users/ywa/Documents/Data/UserReport/de_hei_region_drupal.csv')

de_regions_drupal.head()

# from the second round, use copy of the
de_unique_heis_drupal = drupal_de_heis.copy()

print(len(de_regions_drupal))
print(len(de_unique_heis_drupal))

de_unique_heis_drupal['state'] = de_unique_heis_drupal['provider'].map(de_regions_drupal.set_index('provider')['state'])

print(len(de_regions_drupal))
print(len(de_unique_heis_drupal))

de_unique_heis_drupal.head()

#check on missing values
missing_state_drupal = de_unique_heis_drupal[(de_unique_heis_drupal['state'].isnull())]
missing_state_drupal



In [ ]:
# save the table and maually fill in the states
#missing_state_drupal.to_excel("C:/Users/ywa/Documents/Data/UserReport/MissingStates.xlsx", index=False)

In [ ]:
# import manually filled states
#filled_state_drupal = pd.read_excel('C:/Users/ywa/Documents/Data/UserReport/MissingStates.xlsx')
#filled_state_drupal.head()

#de_unique_heis_drupal = de_unique_heis_drupal.dropna(subset = 'state')
#print(len(de_unique_heis_drupal))
#print(len(filled_state_drupal))

#de_unique_heis_drupal = pd.concat([de_unique_heis_drupal, filled_state_drupal], axis = 0)
#print(len(de_unique_heis_drupal))

de_unique_heis_drupal.head()


In [ ]:
# create unique provider-state combinations for REUSE
provider_state_drupal = de_unique_heis_drupal[['provider', 'state']]

provider_state_drupal.head()

print(len(provider_state_drupal))

print(provider_state_drupal.duplicated().sum())

# drop duplicated combinations
provider_state_drupal = provider_state_drupal.drop_duplicates(keep = 'first')

print(provider_state_drupal.duplicated().sum())


In [ ]:
# save the combinations to a list for reuse ;-)
#provider_state_drupal.to_csv('C:/Users/ywa/Documents/Data/UserReport/de_hei_region_drupal.csv', index = False)


In [ ]:
# aggregate the count by state and provider --> otherwise the count of integration will be doubled /tripled

print(de_unique_heis_drupal['provider'].nunique())

de_heis_unique_agg_drupal = pd.DataFrame(de_unique_heis_drupal.groupby(['state', 'provider']).agg({'count' : 'sum'}))

de_heis_unique_agg_drupal = de_heis_unique_agg_drupal.reset_index()

de_heis_unique_agg_drupal


In [ ]:
# Check on the regional distribution --> interactive maps can be used for a better visualisation
de_unique_heis_drupal.head()

de_regional_count_drupal = pd.DataFrame(de_heis_unique_agg_drupal.groupby('state')['provider', 'count'].agg({'provider' : 'nunique', 'count': 'sum'})).sort_values(by = 'count', ascending = False)

de_regional_count_drupal = de_regional_count_drupal.reset_index()

de_regional_count_drupal.columns = ['Bundesland', 'Anzahl an Hochschulen', 'Anzahl an Hochschulaccounts']
#de_regional_count_drupal.columns = ['federal state', 'num of HEI', 'num of HEI accounts']

print(de_regional_count_drupal['Anzahl an Hochschulen'].sum())
#print(de_regional_count_drupal['num of HEI'].sum())

print(de_regional_count_drupal)

In [ ]:
# create a dataframe with col - state, total num hei, integrated hei
integration_drupal = []

# create loop 
for state in de_regional_count_drupal['Bundesland']:
    state_hei = de_heis_unique_agg_drupal[de_heis_unique_agg_drupal['state'] == state]
    integration = len(state_hei[state_hei['count'] >= 100])
    integration_drupal.append(integration)

de_regional_count_drupal['Anzahl der Integration'] = integration_drupal

# sanity checek
print(de_regional_count_drupal['Anzahl der Integration'].sum())

de_regional_count_drupal

# NRW
#nrw_hei_drupal = de_unique_heis_drupal[de_unique_heis_drupal['state'] == 'Nordrhein-Westfalen']
#print(len(nrw_hei_drupal[nrw_hei_drupal['count'] > 20]))
#print(nrw_hei_drupal[nrw_hei_drupal['count'] > 20])

# Ba-Wü
#ba_wu_drupal = de_unique_heis_drupal[de_unique_heis_drupal['state'] == 'Baden-Württemberg']
#print(len(ba_wu_drupal[ba_wu_drupal['count'] > 20]))

# Bayern
#bayern_drupal = de_unique_heis_drupal[de_unique_heis_drupal['state'] == 'Bayern']
#print(len(bayern_drupal[bayern_drupal['count'] > 20]))

In [ ]:
# sanity check
print(de_regional_count_drupal['Anzahl an Hochschulen'].sum())
print(de_regional_count_drupal['Anzahl an Hochschulaccounts'].sum())

de_regional_count_drupal

# export the Geo-HEI to csv or excel  --> add it to the Drupal excel
#de_regional_count_drupal.to_csv('C:/Users/ywa/Documents/Data/UserReport/Apr_2025_Moodle/de_regional_count_drupal.csv', index = True)
#de_regional_count_drupal.to_excel('C:/Users/ywa/Documents/Data/UserReport/Jan_2026_Moodle/de_regional_count_drupal.xlsx', index = False)

In [ ]:
# pandas Chropleth DE Integration
import geopandas as gpd
german_geojson = gpd.read_file('C:/Users/ywa/Documents/Data/german_states.geojson')
german_geojson = german_geojson.rename(columns={'NAME_1': 'state'}) 

german_map_drupal = german_geojson.merge(de_regional_count_drupal, how="left", left_on="state", right_on = 'Bundesland')

german_map_drupal.plot(column = 'Anzahl der Integration', cmap = 'OrRd', legend = True)




In [ ]:
# Creating interactive folium map 
import geopandas as gpd
import folium

german_geojson = gpd.read_file('C:/Users/ywa/Documents/Data/german_states.geojson')
german_geojson = german_geojson.rename(columns={'NAME_1': 'state'}) 

german_map_drupal = german_geojson.merge(de_regional_count_drupal, how="left", on="state")

german_map_drupal

de_map_drupal = folium.Map(location=[20, 0], zoom_start=2)

folium.Choropleth(geo_data = german_map_drupal, name = 'choropleth', \
                  data = german_map_drupal, columns = ['state', "number of HEIS", 'number of HEI accounts'], key_on = "feature.properties.state", \
                  fill_color = "YlGn", fill_opacity = 0.7, line_opacity = 0.2, highlight=False,\
                  legend_name='Distribution of the German HEIs').add_to(de_map_drupal)

de_geojson_layer_drupal = folium.GeoJson(german_map_drupal, tooltip=folium.GeoJsonTooltip(fields=["state", "number of HEIS", 'number of HEI accounts'],\
                                                                        aliases=["Bundesland:", "Anzahl HEIs:", 'Anzahl HEI Nutzer:']),).add_to(de_map_drupal)


de_map_drupal

In [ ]:
# export HEI data

hei_data_drupal = drupal_unique_heis.reset_index()

hei_data_drupal['state'] = hei_data_drupal['provider'].map(de_regions_drupal.set_index('provider')['state'])
hei_data_drupal['state'] = hei_data_drupal['state'].fillna('Not in Germany')

# sanity check
print(hei_data_drupal['count'].sum())
hei_data_drupal

In [ ]:
# export the data 
#hei_data_drupal.to_excel('C:/Users/ywa/Documents/Data/UserReport/Apr_2025_Moodle/hei_data_drupal.xlsx', index = False)

In [ ]:
# check on data from "others" category
drupal_others = drupal_merge_copy[(drupal_merge_copy['provider_type'] == 1.0)]

drupal_others.head(50)

In [ ]:
# export for manually check
#drupal_others.to_excel("C:/Users/ywa/Documents/Data/UserReport/Apr_2025_Moodle/top50_others_Jan.xlsx", index=False)

In [ ]:
#drupal_commercial_others = drupal_merge_copy[drupal_merge_copy['provider_type'] != 2.0]
#drupal_unique_domains[2] = (drupal_merge_copy.loc[drupal_merge_copy['provider_type'] == 2.0, ['provider']]).nunique()

### Moodle Data Analysis

In [ ]:
# load data
moodle_path = 'C:/Users/ywa/Documents/Data/UserReport/Jan_2026_Moodle/Userreport_Jan2026.xlsx'
moodle = pd.read_excel(moodle_path, sheet_name = 'Sheet1')

In [ ]:
# extract emails
#email_moodle = moodle['E-Mail-Adresse']

# in case of the EN table
email_moodle = moodle['Email address']

print(len(email_moodle))

In [ ]:
# email sanity check
problematic_emails_m = email_moodle[
    ~email_moodle.apply(lambda x: isinstance(x, str)) |
    ~email_moodle.astype(str).str.contains('@', na=False)
]
print(problematic_emails_m)

In [ ]:
# if missing value
email_moodle = email_moodle.dropna()
print(len(email_moodle))

In [ ]:
# domain extraction, output is a df with 'domain' and 'count' columns
moodle_regex = email_moodle.astype(str).str.findall(domain_regex)
moodle_domains = [item[0] if item else None for item in moodle_regex]
moodle_domains = pd.Series(moodle_domains).str.lower()
moodle_domains = pd.DataFrame(moodle_domains.value_counts())
moodle_domains = moodle_domains.reset_index()
moodle_domains.columns = ['domain', 'count']
moodle_domains


In [ ]:
# quick sanity check of the sum
print(moodle_domains['count'].sum())
print(len(email_moodle))


In [ ]:
# merge master list
moodle_merge = moodle_domains.merge(master_list, on = 'domain', how = 'left')
moodle_matched_count = moodle_merge['provider_type'].notna().sum()

print(f'the first round matched record is: {moodle_matched_count}, the total record of the new list is {len(moodle_domains)}, the total record of the master list is {len(master_list)}')

# sanity check
print(moodle_merge['count'].sum())

# check on the merged value
moodle_merge.head(50)

In [ ]:
# standardise DHBW entries
dhbw_regex = 'dhbw'
dhbw_mask = moodle_merge['domain'].str.contains(dhbw_regex)
moodle_merge.loc[dhbw_mask, ['country', 'provider_type', 'provider']] = ['Germany', 2.0, 'DHBW']

# DHBW sanity check
moodle_merge.loc[dhbw_mask]

# change record of f269323f-fa99-4102-81fa-2e6ee79d13e8.ilias and example.com
#unique_domain_copy.loc[unique_domain_copy['domain'] == 'f269323f-fa99-4102-81fa-2e6ee79d13e8.ilias', ['provider']] = 'Landesärztekammer BaWü'
#unique_domain_copy.loc[unique_domain_copy['domain'] == 'example.com', ['provider']] = 'FernUniversität in Hagen'


In [ ]:
# if there're not matched domains
moodle_no_match = moodle_merge[moodle_merge['provider_type'].isna()].copy()
moodle_no_match

In [ ]:
# loop over on the ground truth
moodle_no_match['provider_type'] = moodle_no_match['domain'].apply(lambda x : 0 if x in commercial_providers else None)
moodle_no_match['provider'] = moodle_no_match['provider_type'].apply(lambda x : 0 if x == 0 else None)

# 2nd filter for commercial emails
moodle_mask_commercial = moodle_no_match['domain'].str.contains(commercial_regex)
moodle_no_match.loc[moodle_mask_commercial & moodle_no_match['provider_type'].isna(), ['provider_type', 'provider']] = [0, 0]


for _, row in hei_domains.sort_values(by = 'clean_domain', key = lambda x : x.str.len(), ascending = False).iterrows():
    new_mask_m = moodle_no_match['domain'].str.endswith(row['clean_domain'], na = False)
    moodle_no_match.loc[new_mask_m & moodle_no_match['provider_type'].isna(), ['provider_type', 'provider', 'country']] = [2, row['name'], row['country']]
    
moodle_no_match['provider_type'].fillna(1, inplace = True)
moodle_no_match['provider'].fillna(0, inplace = True)    



In [ ]:
# Sanity check-point --> the not-filled data should be filled 
print(moodle_no_match['provider_type'].isna().sum())

# sanity check
moodle_merge[moodle_merge['domain'] == 'stud.fra-uas.de']
moodle_merge[moodle_merge['domain'] == 'th-nuernberg.de']

In [ ]:
# repair some mis-matches <post hoc>

# Frankfurt University of Applied Sciences
fra_uas_regex = 'fra-uas.de'
fra_uas_mask = moodle_merge['domain'].str.contains(fra_uas_regex)
moodle_merge.loc[fra_uas_mask, ['country', 'provider_type', 'provider']] = ['Germany', 2.0, 'Frankfurt University of Applied Sciences']


moodle_merge.loc[moodle_merge['domain'] == 'stud.hn.de', ['country', 'provider_type', 'provider']] = ['Germany', 2.0, 'Hochschule Niederrhein']
moodle_merge.loc[moodle_merge['domain'] == 'stud.fra-uas.de', ['country', 'provider_type', 'provider']] = ['Germany', 2.0, 'Frankfurt University of Applied Sciences']
moodle_merge.loc[moodle_merge['domain'] == 'stud.ph-gmuend.de', ['country', 'provider_type', 'provider']] = ['Germany', 2.0, 'PH Schwäbisch Gmünd']
moodle_merge.loc[moodle_merge['domain'] == 'stud.hs-furtwangen.de', ['country', 'provider_type', 'provider']] = ['Germany', 2.0, 'Hochschule Furtwangen']
moodle_merge.loc[moodle_merge['domain'] == 'tha.de', ['country', 'provider_type', 'provider']] = ['Germany', 2.0, 'Hochschule Augsburg']
moodle_merge.loc[moodle_merge['domain'] == 'ukbonn.de', ['country', 'provider_type', 'provider']] = ['Germany', 2.0, 'Universitätsklinikum Bonn']
moodle_merge.loc[moodle_merge['domain'] == 'iu.org', ['country', 'provider_type', 'provider']] = ['Germany', 2.0, 'International University']
moodle_merge.loc[moodle_merge['domain'] == 'stud.fra-uas.de', ['country', 'provider_type', 'provider']] = ['Germany', 2.0, 'Frankfurt University of Applied Sciences']
moodle_merge.loc[moodle_merge['domain'] == 'dacs-bund.de', ['country', 'provider_type', 'provider']] = ['Germany', 2.0, 'Hochschule des Bundes für öffentliche Verwaltung']
moodle_merge.loc[moodle_merge['domain'] == 'khm.de', ['country', 'provider_type', 'provider']] = ['Germany', 2.0, 'Kunsthochschule für Medien Köln']
moodle_merge.loc[moodle_merge['domain'] == 'itu.edu.tr', ['country', 'provider_type', 'provider']] = ['Turkey', 2.0, 'Instanbul Technical University']
moodle_merge.loc[moodle_merge['domain'] == 'aau.at' , ['country', 'provider_type', 'provider']] = ['Austria', 2.0, 'Universität Klagenfurt']
moodle_merge.loc[moodle_merge['domain'] == 'hcw.ac.at' , ['country', 'provider_type', 'provider']] = ['Austria', 2.0, 'FH Campus Wien']


In [ ]:
# remained possible edu domains
moodle_edu_candidate = moodle_no_match[(moodle_no_match['provider_type'] == 1) & (moodle_no_match['domain'].str.contains(edu_regex))]
print(moodle_edu_candidate.shape)
print(moodle_edu_candidate)

In [ ]:
# export potential candidates --> do not overwrite
#moodle_edu_candidate.to_csv("C:/Users/ywa/Documents/Data/UserReport/sep_potential_educational_domains_moodle.csv", index=False)
#print("Filtered domains saved for manual review.") 

In [ ]:
# make a copy (separated due to the code repeat)
moodle_merge_copy = moodle_merge.copy()

print(moodle_merge['count'].sum())
print(moodle_merge.shape)

In [ ]:
# Merge maual filtered record with the existing list
#reviewed_edu_moodle = pd.read_csv("C:/Users/ywa/Documents/Data/UserReport/june_potential_educational_domains_moodle.csv", encoding='latin-1', sep = ',')
#print(moodle_merge.shape)

# check dupliactes before merge
#print(moodle_merge_copy[moodle_merge_copy.duplicated(subset = 'domain')])

# concatenate two dataframes
#moodle_merge_copy = pd.concat([moodle_merge_copy, reviewed_edu_moodle], axis = 0)

#print(moodle_merge_copy[moodle_merge_copy.duplicated(subset = 'domain')])

#print(moodle_merge_copy.shape)

#moodle_merge_copy = moodle_merge_copy.drop_duplicates(subset = ['domain', 'count'], keep = 'last').reset_index(drop = True)

print(moodle_merge_copy.shape)

In [ ]:
# NaNs check
print(moodle_merge_copy.isna().sum())

print(moodle_merge_copy[moodle_merge_copy['provider_type'].isna()])

# Relpace NaNs
moodle_merge_copy['country'].fillna('unknown', inplace = True)
moodle_merge_copy['provider_type'].fillna(1.0, inplace = True)
moodle_merge_copy['provider'].fillna(0, inplace = True)

print(moodle_merge_copy.isna().sum())


In [ ]:
# sanity check! 
moodle_merge_copy['count'].sum()

In [ ]:
# Update the master list if there are domain updates
print(len(master_list))
print(len(moodle_merge_copy))

# update master list
master_update_m = pd.concat([master_list, moodle_merge_copy]).drop_duplicates(subset = ['domain'], keep = 'first').reset_index(drop = True)
print(len(master_update_m))

# for excel export
#master_update_m.to_excel("C:/Users/ywa/Documents/Data/UserReport/MasterList_Update.xlsx", index=False)

# for csv export
#master_update_m.to_csv("C:/Users/ywa/Documents/Data/UserReport/MasterList_Update.csv", index=False)


## Analysis!

In [ ]:
# count the number of unique domains
print(f'There are {len(moodle_merge_copy)} unique domains on Moodle')
moodle_merge_copy.head(15)

In [ ]:
# number of accounts based on the provider type
moodle_ptype_count = pd.DataFrame(moodle_merge_copy.groupby('provider_type')['count'].sum().sort_values(ascending = False))

print(moodle_ptype_count)

# adjust index based on the output
moodle_ptype_count.index = ['Commercial', 'Others', 'HEIs']
moodle_ptype_count['ratio'] = round((moodle_ptype_count['count'] / moodle_ptype_count['count'].sum()) * 100, 2)

moodle_ptype_count['count'].plot.bar(rot = 0)

print(moodle_ptype_count)
print(moodle_ptype_count['count'].sum())

In [ ]:
# counting the number of domains in different categories
moodle_unique_domains = moodle_merge_copy['provider_type'].value_counts()
moodle_unique_domains = pd.DataFrame(moodle_unique_domains)

print(moodle_unique_domains)

# adjust index based on the code
moodle_unique_domains.index = ['Other', 'HEIs', 'Commercial']
moodle_unique_domains.columns = ['count']
moodle_unique_domains['ratio'] = round((moodle_unique_domains['count'] / moodle_unique_domains['count'].sum()) * 100, 2)
print(moodle_unique_domains)

moodle_unique_domains['count'].plot.bar(rot = 0)

print(moodle_unique_domains['count'].sum())

In [ ]:
# HEI distribution in DACH region
moodle_de_heis = moodle_merge_copy[(moodle_merge_copy['country'] == 'Germany') & (moodle_merge_copy['provider_type'] == 2)]
moodle_at_heis = moodle_merge_copy[(moodle_merge_copy['country'] == 'Austria') & (moodle_merge_copy['provider_type'] == 2)]
moodle_ch_heis = moodle_merge_copy[(moodle_merge_copy['country'] == 'Switzerland') & (moodle_merge_copy['provider_type'] == 2)]
                                                                                      
print(f'There are {moodle_de_heis.shape[0]} unique domains from german HEIs')
print(f'There are {moodle_at_heis.shape[0]} unique domains from austrian HEIs')
print(f'There are {moodle_ch_heis.shape[0]} unique domains from swiss HEIs')

In [ ]:
# The number of accounts in HEIs
moodle_unique_heis = moodle_merge_copy[moodle_merge_copy['provider_type'] == 2.0]
moodle_unique_heis = pd.DataFrame(moodle_unique_heis.groupby('provider')['count'].sum().sort_values(ascending = False))
moodle_unique_heis.head(50)


In [ ]:
# Select HEI with more than 100 accounts
moodle_integration_edu = moodle_unique_heis[moodle_unique_heis['count'] >= 100]
print(f'There are {len(moodle_integration_edu)} HEIs that have possibly integrated KIC content in the currirulum, which is {len(moodle_integration_edu) /len(moodle_unique_heis) * 100} percent of the total HEIs on KIC')


In [ ]:
# Geographic distribution of the HEI accounts
moodle_hei_country = moodle_merge_copy[(moodle_merge_copy['provider_type'] == 2.0)]
moodle_hei_ncountry = moodle_hei_country['country'].nunique()
print(f'The HEI users from {moodle_hei_ncountry} countries have registerred on our platform.')

moodle_hei_countrycount = pd.DataFrame(moodle_hei_country.groupby('country')['count'].sum().sort_values(ascending = False))
print(moodle_hei_countrycount.head(10))


In [ ]:
# Geographic distribution of the German HEIS (federal states)

# match federal states by provider

####### with german_heis list#######
#de_unique_heis_drupal = drupal_de_heis.merge(german_heis[['Name', 'Region']], left_on = 'provider', right_on = 'Name',  how = 'left').rename(columns = {'Region' : 'state'})

#de_unique_heis_drupal = de_unique_heis_drupal.drop('Name', axis = 1)

#de_unique_heis_drupal['state'].isnull().sum()

###### with the provider-state combination from the first round ######
de_regions_drupal = pd.read_csv('C:/Users/ywa/Documents/Data/UserReport/de_hei_region_drupal.csv')

de_regions_drupal.head()
#de_regions_drupal.shape

moodle_de_heis['state'] = moodle_de_heis['provider'].map(de_regions_drupal.set_index('provider')['state'])

moodle_de_heis.head()

#check on missing values
missing_state_moodle = moodle_de_heis[(moodle_de_heis['state'].isnull())]
missing_state_moodle


In [ ]:
# manually fill in the state info directly in the moodle_de_heis
#moodle_de_heis.loc[moodle_de_heis['provider'] == 'Landesärztekammer BaWü', 'state'] = 'Baden-Württemberg'
#moodle_de_heis['state'] = moodle_de_heis['state'].fillna('Hessen')
moodle_de_heis.loc[moodle_de_heis['provider'] == 'Hochschule für Musik Trossingen', 'state'] = 'Baden-Württemberg'

#moodle_de_heis.loc[moodle_de_heis['provider'] == 'Frankfurt University of Applied Sciences', 'state'] = 'Hessen'
#moodle_de_heis.loc[moodle_de_heis['provider'] == 'International University', 'state'] = 'Thüringen'
#moodle_de_heis.loc[moodle_de_heis['provider'] == 'Universitätsklinikum Bonn', 'state'] = 'Nordrhein-Westfalen'
#moodle_de_heis.loc[moodle_de_heis['provider'] == 'Humanistische Hochschule Berlin', 'state'] = 'Berlin'

# sanity check
moodle_de_heis['state'].isnull().sum()

In [ ]:
# sanity check Frankfurt University of Applied Science
moodle_de_heis[moodle_de_heis['provider'] == 'Frankfurt University of Applied Sciences']

In [ ]:
# update the regional information --> use the drupal list instead (can be updated manually)
#moodle_de_heis[['provider', 'state']].drop_duplicates().to_csv('C:/Users/ywa/Documents/Data/UserReport/de_hei_region_moodle.csv', index = False)

In [ ]:
# aggregate the count by state and provider --> otherwise the count of integration will be doubled /tripled
de_heis_unique_agg_moodle = pd.DataFrame(moodle_de_heis.groupby(['state', 'provider']).agg({'count' : 'sum'}))

de_heis_unique_agg_moodle = de_heis_unique_agg_moodle.reset_index()

de_heis_unique_agg_moodle


In [ ]:
# Check on the regional distribution --> interactive maps can be used for a better visualisation

de_regional_count_moodle = pd.DataFrame(de_heis_unique_agg_moodle.groupby('state')['provider', 'count'].agg({'provider' : 'nunique', 'count': 'sum'})).sort_values(by = 'count', ascending = False)

de_regional_count_moodle = de_regional_count_moodle.reset_index()

de_regional_count_moodle

de_regional_count_moodle.columns = ['Bundesland', 'Anzahl an Hochschulen', 'Anzahl an Hochschulaccounts']

#de_regional_count_moodle.columns = ['federal state', 'num of HEI', 'num of HEI accounts']

print(de_regional_count_moodle)

# sanity check
print(de_regional_count_moodle['Anzahl an Hochschulen'].sum())
print(de_regional_count_moodle['Anzahl an Hochschulaccounts'].sum())


In [ ]:
# adding column of possible HEI with integration
integration_moodle = []

# create loop 
for state in de_regional_count_moodle['Bundesland']:
    state_hei = de_heis_unique_agg_moodle[de_heis_unique_agg_moodle['state'] == state]
    integration = len(state_hei[state_hei['count'] >= 100])
    integration_moodle.append(integration)

de_regional_count_moodle['Anzahl der Integration'] = integration_moodle

de_regional_count_moodle


In [ ]:
# sanity check
print(de_regional_count_moodle['Anzahl an Hochschulaccounts'].sum())
print(de_regional_count_moodle['Anzahl an Hochschulen'].sum())
print(de_regional_count_moodle['Anzahl der Integration'].sum())

print(len(de_heis_unique_agg_moodle[de_heis_unique_agg_moodle['count'] >= 100]))

# export table to excel
# export the Geo-HEI to csv or excel  --> add it to the Drupal excel
#de_regional_count_moodle.to_excel('C:/Users/ywa/Documents/Data/UserReport/Feb_2025_Moodle/de_regional_count_moodle.xlsx', index = False)
#de_regional_count_moodle.to_excel('C:/Users/ywa/Documents/Data/UserReport/Jan_2026_Moodle/de_regional_count_moodle.xlsx', index = False)

In [ ]:
# pandas Chropleth DE Integration

import geopandas as gpd
german_geojson = gpd.read_file('C:/Users/ywa/Documents/Data/german_states.geojson')
german_geojson = german_geojson.rename(columns={'NAME_1': 'state'}) 

german_map_moodle = german_geojson.merge(de_regional_count_moodle, how="left", left_on="state", right_on = 'Bundesland')

german_map_moodle.plot(column = 'Anzahl der Integration', cmap = 'OrRd', legend = True)

In [ ]:
# Creating interactive folium map 
import geopandas as gpd
import folium

german_geojson = gpd.read_file('C:/Users/ywa/Documents/Data/german_states.geojson')
german_geojson = german_geojson.rename(columns={'NAME_1': 'state'}) 

german_map_moodle = german_geojson.merge(de_regional_count_moodle, how="left", on="state")

german_map_moodle

de_map_moodle = folium.Map(location=[20, 0], zoom_start=2)

folium.Choropleth(geo_data = german_map_moodle, name = 'choropleth', \
                  data = german_map_moodle, columns = ['state', "number of HEIS", 'number of HEI accounts'], key_on = "feature.properties.state", \
                  fill_color = "YlGn", fill_opacity = 0.7, line_opacity = 0.2, highlight=False,\
                  legend_name='Distribution of the German HEIs').add_to(de_map_moodle)

de_geojson_layer_moodle = folium.GeoJson(german_map_moodle, tooltip=folium.GeoJsonTooltip(fields=["state", "number of HEIS", 'number of HEI accounts'],\
                                                                        aliases=["Bundesland:", "Anzahl HEIs:", 'Anzahl HEI Nutzer:']),).add_to(de_map_moodle)


de_map_moodle

In [ ]:
# export moodle hei data
hei_data_moodle = moodle_unique_heis.reset_index()

hei_data_moodle['state'] = hei_data_moodle['provider'].map(de_regions_drupal.set_index('provider')['state'])
hei_data_moodle['state'] = hei_data_moodle['state'].fillna('Not in Germany')

# sanity check
print(hei_data_moodle['count'].sum())
hei_data_moodle

In [ ]:
# export the data
#hei_data_moodle.to_excel('C:/Users/ywa/Documents/Data/UserReport/Feb_2026_Moodle/hei_data_moodle.xlsx', index = False)

In [ ]:
# check on data from "others" category
moodle_others = moodle_merge_copy[(moodle_merge_copy['provider_type'] == 1.0)]

moodle_others.head(50)

In [ ]:
# export others category to excel
#moodle_others.to_excel('C:/Users/ywa/Documents/Data/UserReport/Feb_2026_Moodle/others_moodle.xlsx', index = False)

### Partner Moodle User
* the goal is to exclude the duplicate users and then merge with the product Moodle data

In [ ]:
# read enrolment export from moodle partner

enrol_path = 'C:/Users/ywa/Documents/Data/Partner_Moodle_enrollment/Jan2026/institution_enrol.xlsx'

enrol_df = pd.read_excel(enrol_path)

enrol_df.head()


In [ ]:
# inspect categories

enrol_df['Category name'].unique()

exclusion_cat = ['Testing', 'intern', 'Data Literacy', 'KI und Ethik', 'Über KI ', 'Studieren', 'Community', 'Schule', 'Verwaltung', 'Erwachsenenbildung', 'Kursbibliothek', 'Testing']

print(len(enrol_df))

# drop rows based on the category 

enrol_df = enrol_df[~enrol_df['Category name'].isin(exclusion_cat)]

print(len(enrol_df))

enrol_df['Category name'].unique()

In [ ]:
# string preprocessing

enrol_df['Category name'] = enrol_df['Category name'].str.strip()

enrol_df['Name'] = enrol_df['Name'].str.lower().str.strip()

In [ ]:
# create a dictionary that maps the categories

cat_dic = {'FernUniversität in Hagen' : ['FernUni Hagen', 'FeUW Institut für wissenschaftliche Weiterbildung', 'LG Bildungstheorie. u. Medienpädagogik', 'OpenEU', 'Kopien aus Kursbibliothek', 'FeU-Zertifikatsstudium'],\
           'DHBW' : ['DHBW Heilbronn', 'DHBW Stuttgart', 'DHBW Mosbach', 'DHBW Mannheim', 'DHBW-weite Angebote', 'DHBW Villingen-Schwenningen bulk action selection','DHBW Villingen-Schwenningen'],\
           'Hochschule Heilbronn' : ['Fakultät Informatik', 'Innovation Lab - WS 25/26', 'Fakultät Technik', 'Fakultät International Business', 'Fakultät Wirtschaft', 'Fakultät Management und Vertrieb', 'Allgemeine KI-Kurse HHN'],\
           'AI Traqc' : ['Basics KI', 'KI Recht und Ethik', 'Branchenspezifische Kurse', 'Prompting und Chatbots', 'Prompting und Chatbots', 'Business Innovation - KI neu im Unternehmen'],\
           'Universität Wuppertal' : ['Bergische Universität Wuppertal', 'School of Education'],\
           'Karlsruher Institut für Technologie (KIT)' : ['KIT - Karlsruher Institut für Technologie', 'KOALA'],\
           'Hochschule Düsseldorf' : ['Fachbereich Medien', 'Künstliche Intelligenz sozialverantwortlich gestalten'],\
           'Verband kommunaler Unternehmen e.V.' : ['Industrie 4.0'],\
           'Frankfurt University of Applied Sciences' : ['DigiKomT'], \
           "Hafencity Universität Hamburg" : ['HafenCity Universität Hamburg']
          }

# reverse the dictionary for mapping

cat_dic_reverse = {v : k for k, values in cat_dic.items() for v in values }

enrol_df['Category name'] = enrol_df['Category name'].replace(cat_dic_reverse)

enrol_df.head()


In [ ]:
print(len(enrol_df))

exclusion_name = ['manual enrolments', 'auto enrol  (student)']

#  exclude rows of non-LTI indidviduals
enrol_df = enrol_df[~enrol_df['Name'].isin(exclusion_name)]

#print(enrol_df['Name'].unique())

print(len(enrol_df))

enrol_df.head()


In [ ]:
# to filter out unique users from each institution (categories)

user_df = enrol_df.drop_duplicates(subset = ['Full name with link', 'Email address'], keep = 'first' )

print(len(user_df))

In [ ]:
# exclude dupliactes from product Moodle --> exclude product moodle email from partner email list

product_df = user_df[~user_df['Email address'].isin(moodle['Email address'])]

print(len(product_df))


In [ ]:
# calculate user number per institution (category)

partner_df = pd.DataFrame(product_df.groupby(['Category name'])['Email address'].count())

partner_df = partner_df.reset_index()

partner_df.columns = ['provider', 'count']

partner_df.head()

In [ ]:
# merge with hei_data_moodle dataframe on "provider" --> provider type is needed due to company users

# merge with master list 

master_simplified = master_list.drop_duplicates(subset = ['provider', 'country', 'provider_type']).drop('domain', axis = 1)

partner_merge = partner_df.merge(master_simplified, on = 'provider', how = 'left')

partner_merge


In [ ]:
# change the provider_type and merge with HEI table

partner_merge.loc[partner_merge['provider'] == 'Frankfurt University of Applied Sciences', ['country', 'provider_type']] = ['Germany', 2.0]
partner_merge.loc[partner_merge['provider'] == 'Hochschule für Finanzwirtschaft & Management GmbH', ['country', 'provider_type']] = ['Germany', 1.0]
partner_merge.loc[partner_merge['provider'] == 'LÄK Hessen', ['country', 'provider_type']] = ['Germany', 1.0]
partner_merge.loc[partner_merge['provider'] == 'SCHUNK', ['country', 'provider_type']] = ['Germany', 1.0]

partner_merge


In [ ]:
# merge HEI moodle

hei_partner = partner_merge[partner_merge['provider_type'] == 2.0]

hei_data_total_moodle = hei_data_moodle.merge(hei_partner, how = 'left', on = 'provider')

hei_data_total_moodle = hei_data_total_moodle.drop(['country', 'provider_type'], axis = 1)

check_idx = hei_data_total_moodle[hei_data_total_moodle['count_y'].notnull()].index

hei_data_total_moodle['count_y'] = hei_data_total_moodle['count_y'].fillna(0)

hei_data_total_moodle['count_x'] = hei_data_total_moodle['count_x'] + hei_data_total_moodle['count_y']

# Sanity check
hei_data_total_moodle.iloc[check_idx]


In [ ]:
# drop count_y and rename count_x
hei_data_total_moodle = hei_data_total_moodle.drop('count_y', axis = 1)

hei_data_total_moodle.rename(columns = {'count_x' : 'count'}, inplace = True)

hei_data_total_moodle = hei_data_total_moodle.sort_values(by = ['count'], ascending = False).reset_index()

hei_data_total_moodle.drop('index', axis = 1, inplace = True)

hei_data_total_moodle.head()


In [ ]:
# Select HEI with more than 100 accounts
moodle_total_integration_edu = hei_data_total_moodle[hei_data_total_moodle['count'] >= 100]
print(f'There are {len(moodle_total_integration_edu)} HEIs that have possibly integrated KIC content in the currirulum, which is {len(moodle_total_integration_edu) /len(hei_data_total_moodle) * 100} percent of the total HEIs on KIC')


In [ ]:
# export to excel

#hei_data_total_moodle.to_excel('C:/Users/ywa/Documents/Data/UserReport/Jan_2026_Moodle/hei_data_total.xlsx', index = False)

In [ ]:
# State HEI total

# exclude 'Not in Germany'
moodle_de_heis_total = hei_data_total_moodle[~hei_data_total_moodle['state'].isin(['Not in Germany'])]

# aggregate the count by state and provider --> otherwise the count of integration will be doubled /tripled
de_heis_total_unique_agg_moodle = pd.DataFrame(moodle_de_heis_total.groupby(['state', 'provider']).agg({'count' : 'sum'}))

de_heis_total_unique_agg_moodle = de_heis_total_unique_agg_moodle.reset_index()

de_heis_total_unique_agg_moodle


In [ ]:
# Check on the regional distribution total moodle users

de_regional_count_total_moodle = pd.DataFrame(de_heis_total_unique_agg_moodle.groupby('state')['provider', 'count'].agg({'provider' : 'nunique', 'count': 'sum'})).sort_values(by = 'count', ascending = False)

de_regional_count_total_moodle = de_regional_count_total_moodle.reset_index()

de_regional_count_total_moodle

de_regional_count_total_moodle.columns = ['Bundesland', 'Anzahl an Hochschulen', 'Anzahl an Hochschulaccounts']

#de_regional_count_total_moodle.columns = ['federal state', 'num of HEI', 'num of HEI accounts']

print(de_regional_count_total_moodle)

# sanity check
print(de_regional_count_total_moodle['Anzahl an Hochschulen'].sum())
print(de_regional_count_total_moodle['Anzahl an Hochschulaccounts'].sum())

In [ ]:
# adding column of possible HEI with integration
integration_total_moodle = []

# create loop 
for state in de_regional_count_total_moodle['Bundesland']:
    state_hei = de_heis_total_unique_agg_moodle[de_heis_total_unique_agg_moodle['state'] == state]
    integration = len(state_hei[state_hei['count'] >= 100])
    integration_total_moodle.append(integration)

de_regional_count_total_moodle['Anzahl der Integration'] = integration_total_moodle

de_regional_count_total_moodle

In [ ]:
# pandas Chropleth DE Integration

import geopandas as gpd
german_geojson = gpd.read_file('C:/Users/ywa/Documents/Data/german_states.geojson')
german_geojson = german_geojson.rename(columns={'NAME_1': 'state'}) 

german_map_total_moodle = german_geojson.merge(de_regional_count_total_moodle, how="left", left_on="state", right_on = 'Bundesland')

german_map_total_moodle.plot(column = 'Anzahl der Integration', cmap = 'OrRd', legend = True)

In [ ]:
# export to excel

#de_regional_count_total_moodle.to_excel('C:/Users/ywa/Documents/Data/UserReport/Jan_2026_Moodle/de_regional_count_total_moodle.xlsx', index = False)

In [ ]:
# merge others Moodle
hei_partner = partner_merge[partner_merge['provider_type'] == 1.0]
hei_partner

## Additional request: HEI Integration-based Partner Analysis

In [ ]:
# top 50 HEIs on Moodle with regional information
top50_moodle = moodle_unique_heis.head(50)
top50_moodle['state'] = top50_moodle.index.map(de_regions_drupal.set_index('provider')['state'])

top50_moodle.head()

In [ ]:
# export top 50 HEIs on Moodle
#top50_moodle.to_excel('C:/Users/ywa/Documents/Data/UserReport/June_2025_Moodle/top50_HEIS.xlsx', index = True)